In [1]:
import Pkg
Pkg.add("QuantumToolbox")

   Resolving package versions...
     Project No packages added to or removed from `C:\Users\Vedant\.julia\environments\v1.12\Project.toml`
    Manifest No packages added to or removed from `C:\Users\Vedant\.julia\environments\v1.12\Manifest.toml`


In [3]:
using QuantumToolbox
using BenchmarkTools

function benchmark_jch_toolbox(N_values)
    times = Float64[]
    
    # Physics Parameters
    ω = 1.0
    g = 0.05
    J = 0.05
    tlist = range(0, 50, length=200)
    
    println("--- Starting QuantumToolbox.jl Benchmark ---")
    
    for N in N_values
        # 1. Define Basic Operators
        # Note: QuantumToolbox uses the same logic as QuTiP
        # Order: [Cavity A, Atom A, Cavity B, Atom B]
        
        a  = destroy(N)
        id_cav = qeye(N)
        
        sm = sigmam()
        sz = sigmaz()
        id_atom = qeye(2)
        
        # 2. Construct Composite Operators
        # System A
        a1  = tensor(a, id_atom, id_cav, id_atom)
        sm1 = tensor(id_cav, sm, id_cav, id_atom)
        sz1 = tensor(id_cav, sz, id_cav, id_atom)
        
        # System B
        a2  = tensor(id_cav, id_atom, a, id_atom)
        sm2 = tensor(id_cav, id_atom, id_cav, sm)
        sz2 = tensor(id_cav, id_atom, id_cav, sz)
        
        # 3. Build Hamiltonian
        # Note: In Julia, we use ' for dagger (conjugate transpose)
        H = ω * a1' * a1 + 0.5 * ω * sz1 + g * (a1' * sm1 + a1 * sm1') +
            ω * a2' * a2 + 0.5 * ω * sz2 + g * (a2' * sm2 + a2 * sm2') -
            J * (a1' * a2 + a1 * a2')
            
        # 4. Initial State
        # Atom A excited (0), Atom B ground (1) -> QuTiP convention basis(2,0) is up
        psi0 = tensor(basis(N,0), basis(2,0), basis(N,0), basis(2,1))
        
        # 5. Measure Time
        # We assume the Hamiltonian is constant (TimeIndependent)
        print("N=$N... ")
        dt = @elapsed mesolve(H, psi0, tlist)
        
        push!(times, dt)
        println("$dt seconds")
    end
    
    return times
end

# Define N values to test
N_list = [3, 5, 8, 10, 12, 15, 18, 20]

# Run Benchmark
toolbox_times = benchmark_jch_toolbox(N_list)
println("QuantumToolbox Times: ", toolbox_times)

--- Starting QuantumToolbox.jl Benchmark ---

[sesolve] 100%|███████████████████████████| Time: 0:00:00 ( 0.00 ns/it)
[sesolve] 100%|███████████████████████████| Time: 0:00:00 ( 0.00 ns/it)
[sesolve] 100%|███████████████████████████| Time: 0:00:00 ( 5.00 μs/it)
[sesolve] 100%|███████████████████████████| Time: 0:00:00 ( 5.00 μs/it)
[sesolve] 100%|███████████████████████████| Time: 0:00:00 (10.00 μs/it)
[sesolve] 100%|███████████████████████████| Time: 0:00:00 (10.00 μs/it)
[sesolve] 100%|███████████████████████████| Time: 0:00:00 (15.00 μs/it)
[sesolve] 100%|███████████████████████████| Time: 0:00:00 (15.00 μs/it)



N=3... 0.0013074 seconds
N=5... 0.0006501 seconds
N=8... 0.0009523 seconds
N=10... 0.0014473 seconds
N=12... 0.0027725 seconds
N=15... 0.0030302 seconds
N=18... 0.004251 seconds
N=20... 0.003961 seconds
QuantumToolbox Times: [0.0013074, 0.0006501, 0.0009523, 0.0014473, 0.0027725, 0.0030302, 0.004251, 0.003961]
